## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-22-58-49 +0000,nypost,Dozens infected in massive measles outbreak at...,https://nypost.com/2026/02/16/us-news/largest-...
1,2026-02-16-22-56-32 +0000,nypost,Students turn on high school seniors caught in...,https://nypost.com/2026/02/16/us-news/students...
2,2026-02-16-22-56-23 +0000,nypost,Indebted DNC bought Kamala Harris donor list f...,https://nypost.com/2026/02/16/us-news/indebted...
3,2026-02-16-22-54-41 +0000,nyt,2 Killed in Shooting at High School Hockey Gam...,https://www.nytimes.com/2026/02/16/us/hockey-g...
4,2026-02-16-22-48-12 +0000,nyt,"Japan Wins Pairs Figure Skating, Knocking Chin...",https://www.nytimes.com/2026/02/16/world/japan...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2507/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,23
51,guthrie,11
376,new,11
70,iran,8
621,up,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
241,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,59
86,2026-02-16-18-07-29 +0000,wapo,"Popular anger burns in Iran after crackdown, a...",https://www.washingtonpost.com/world/2026/02/1...,50
213,2026-02-16-05-39-01 +0000,bbc,Trump's new world order has become real and Eu...,https://www.bbc.com/news/articles/cddn002g6qzo...,49
98,2026-02-16-17-26-59 +0000,nypost,Trump tears into Gavin Newsom as California go...,https://nypost.com/2026/02/16/us-news/trump-sa...,46
78,2026-02-16-18-29-22 +0000,nypost,Trump says Board of Peace will discuss how to ...,https://nypost.com/2026/02/16/us-news/trump-sa...,45


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
241,59,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
41,41,2026-02-16-21-16-21 +0000,nypost,Walmart backpack is a ‘very promising lead’ in...,https://nypost.com/2026/02/16/us-news/walmart-...
97,37,2026-02-16-17-28-44 +0000,nypost,Kremlin slams claims it killed top Putin criti...,https://nypost.com/2026/02/16/world-news/kreml...
156,35,2026-02-16-13-44-07 +0000,cbc,"Dana Eden, co-creator of Israeli TV series Teh...",https://www.cbc.ca/news/world/dana-eden-israel...
106,34,2026-02-16-16-57-58 +0000,cbc,"Obama clarifies comments on aliens being real,...",https://www.cbc.ca/news/world/obama-aliens-rea...
42,33,2026-02-16-21-13-36 +0000,nypost,California gubernatorial hopeful Eric Swalwell...,https://nypost.com/2026/02/16/us-news/eric-swa...
62,32,2026-02-16-19-45-37 +0000,nypost,Canada announces Janice Charette as new chief ...,https://nypost.com/2026/02/16/business/canada-...
29,30,2026-02-16-21-43-26 +0000,nypost,"School secretary, 31, accused of having sex wi...",https://nypost.com/2026/02/16/us-news/indiana-...
45,28,2026-02-16-20-55-26 +0000,nypost,Long Island Rep Laura Gillen faces uphill batt...,https://nypost.com/2026/02/16/us-news/long-isl...
86,27,2026-02-16-18-07-29 +0000,wapo,"Popular anger burns in Iran after crackdown, a...",https://www.washingtonpost.com/world/2026/02/1...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
